In [ ]:
# Emotion Detection CNN with Activation Map Visualization

## Step 1: Import Libraries
import numpy as np
from keras.models import Sequential, model_from_json, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
import matplotlib.pyplot as plt

## Step 2: Create Dummy Dataset (Grayscale 48x48)
x_train = np.random.rand(100, 48, 48, 1)  # 100 fake grayscale images
y_train = np.random.randint(0, 7, 100)    # 7 emotion classes
y_train = to_categorical(y_train, 7)

## Step 3: Define CNN Model
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1), name='conv1'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu', name='conv2'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # 7 classes

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## Step 4: Train the Model (only for 3 epochs)
model.fit(x_train, y_train, epochs=3, batch_size=16)

## Step 5: Save Model (JSON + Weights)
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_weights.h5")
print("✅ Model saved.")

## Step 6: Create Test Image
test_img = np.random.rand(1, 48, 48, 1).astype('float32')

## Step 7: Visualize Activation Maps
layer_outputs = [layer.output for layer in model.layers if 'conv' in layer.name]
activation_model = Model(inputs=model.input, outputs=layer_outputs)

activations = activation_model.predict(test_img)
first_layer_activation = activations[0]
num_filters = first_layer_activation.shape[-1]

plt.figure(figsize=(20, 5))
for i in range(min(8, num_filters)):
    ax = plt.subplot(1, 8, i+1)
    plt.imshow(first_layer_activation[0, :, :, i], cmap='viridis')
    plt.axis('off')
plt.suptitle("Activation Maps from Conv Layer 1")
plt.show()
